In [7]:
import cv2
import numpy as np
import os


In [8]:
img = cv2.imread('./images/final_exam/Templates/Template-2.png', cv2.IMREAD_GRAYSCALE)
cap = cv2.VideoCapture('./videos/final_exam/Dataset-2/left_output.avi')
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)

detect = cv2.xfeatures2d.SIFT_create() # การตรวจจับคุณสมบัติโดยใช้ SIFT 
ref_kpts, ref_desc = detect.detectAndCompute(img, None) #ตรวจจับและคำนวณโดยใช้ภาพ img
index_params = dict(algorithm=0, trees=5)
search_params = dict()
flann = cv2.FlannBasedMatcher(index_params, search_params) #ใช้คำสั่ง จับคู่


while True : 
    _, frame = cap.read()
    grayframe = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    grayframe_kpts, grayframe_desc = detect.detectAndCompute(grayframe, None) # ตรวจจับและคำนวณโดยใช้วีดีโอที่เป็นสีเทา
    matches = flann.knnMatch(ref_desc, grayframe_desc,k=2) # จับคู่ ref_desc(ภาพ) และ grayframe_desc(VDO)

    goodpoints = []
    goodpointslist = []
    
    for m,n in matches :
        if m.distance < 0.7*n.distance :
            goodpoints.append(m)
            goodpointslist.append([m])

    if len(goodpoints) >14:
        queryIdx = np.float32([ref_kpts[m.queryIdx].pt for m in goodpoints]).reshape(-1,1,2)
        trainIdx = np.float32([grayframe_kpts[m.trainIdx].pt for m in goodpoints]).reshape(-1, 1, 2)
        matrix, inlier_masks = cv2.findHomography(queryIdx, trainIdx, cv2.RANSAC, 3.0) #หาค่า Homography
        matches_mask = inlier_masks.ravel().tolist()
        h, w = img.shape[:2]
        phase = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1, 1, 2)
        margin = cv2.perspectiveTransform(phase, matrix)
        detected = cv2.polylines(frame, [np.int32(margin)], True, (0, 255, 80), 2,cv2.LINE_AA)
        drawmatch_img = cv2.drawMatchesKnn(img, ref_kpts,detected,grayframe_kpts, goodpointslist, None, flags=2, matchesMask=inlier_masks)
        cv2.imshow("drawmatch_img",drawmatch_img)    
    
    cv2.imshow("framet",frame)

    if cv2.waitKey(33) & 0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()